# **Описание**
Бот использует нейронные сети, чтобы научиться распознавать паттерны в сообщениях пользователя, и отвечать соответствующим образом.
Бот анализирует сообщения на английском языке.


Ответы прописаны заранее для описанных классов сообщений. В качестве ответа вы получаете одно из подготовленных сообщений на соответствующую тематику.

---
Классы создаются вручную в файле json.
В загружаемом файле описаны классы:
* Hello - приветствия
* Bot - вопросы о боте
* Age - вопросы о возрасте
* Annoying - сообщения по типу "ты меня достал"
* Question - просьба ответить на вопрос
* Help - просьба о помощи
* Exit - прощание
* SupvervLearn - машинное обучение
* NN - нейронные сети
* Bully - оскорбления
* Ticket - оценка ответа

Внутри каждого класса задается набор сообщений, которые выступают в качестве паттернов. Из этих паттернов создается обучающая выборка.

Модель учится распознавать паттерны в полученном сообщении, после чего она относит сообщение к заданному классу и отвечает одним из вариантов заранее заданных ответов для класса.


---
При желании можно дополнить json файл своими классами или дописать больше примеров паттернов в уже существующие, это должно повысить аккуратность модели.


---






---



> # <font color='red'> Перед использованием бота необходимо целиком запустить весь ноутбук </font>


---







# **Модель**

Подключаем Google Drive, чтобы скачать обучающий датасет.

In [1]:
!pip install gdown

import gdown

url = 'https://drive.google.com/uc?id=1p7-hWXUiMLftwjFHpVK0GXqaikT5A6md'
output = 'Train_Bot.json'

gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1p7-hWXUiMLftwjFHpVK0GXqaikT5A6md
To: /content/Train_Bot.json
100%|██████████| 6.07k/6.07k [00:00<00:00, 5.38MB/s]


'Train_Bot.json'

---
### Подключаем библиотеки
---



In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

---
### Предобработка данных
---



In [3]:
lemmatizer = WordNetLemmatizer()


words=[]
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']
data_file = open("Train_Bot.json").read()
intents = json.loads(data_file)

nltk.download('punkt')
nltk.download('wordnet')

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #токенизируем предложения из pattern
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #добавляем предложения в категорию 'tag'
        documents.append((w, intent['tag']))

        # добавляем тег в класс
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# получаем список уникальных лемматизированных слов из pattern
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# выделяем уникальные классы
classes = sorted(list(set(classes)))


#------------------------------------------------------------------------------
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
#------------------------------------------------------------------------------

# сохраняем полученные данные
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

131 documents
11 classes ['Age', 'Annoying', 'Bot', 'Bully', 'Exit', 'Hello', 'Help', 'NN', 'Question', 'SupvervLearn', 'Ticket']
155 unique lemmatized words ["'d", "'re", 'a', 'able', 'about', 'activation', 'ada', 'adam', 'afternoon', 'age', 'am', 'an', 'ann', 'annoy', 'annoying', 'answer', 'are', 'artificial', 'assistance', 'backward', 'bad', 'bagging', 'bayes', 'best', 'better', 'bloody', 'boosting', 'bot', 'buddy', 'can', 'classification', 'create', 'cross', 'cya', 'day', 'deep', 'define', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'explain', 'forest', 'forward', 'function', 'give', 'good', 'goodbye', 'gradient', 'great', 'greet', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'how', 'howdy', 'hyper', 'i', 'imputer', 'incredibly', 'intelligence', 'irritating', 'is', 'jerk', 'joke', 'just', 'knn', 'know', 'later', 'layer', 'learning', 'leaving', 'like', 'logistic', 'lot', 'machine', 'me', 'ml', 'much', 'my', 'naive', 'nb', 'need', 'net', 'network', 'neur

---
### Создаем обучающую выборку
---



In [5]:
training = []

output_empty = [0] * len(classes)

for doc in documents:

    bag = []

    # список слов из pattern
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # добавляем слово в мешок слов, если совпадение нашлось в текущем pattern
    for w in words:
      if w in pattern_words:
        bag.append(1)
      else:
        bag.append(0)

    # помечаем текущий tag 1, остальные 0
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])



random.shuffle(training)
training = np.array(training)

X_train = list(training[:,0])
y_train = list(training[:,1])

print("Training data created")

Training data created


<ipython-input-5-c7fe597d34e8>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


---
### Создаем нейронную сеть
---



In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# softmax чтобы в сумме получали 1, т.е. имеем вероятности
model.add(Dense(len(y_train[0]), activation='softmax'))

# Стохастический градиентный спуск
sgd = SGD(learning_rate=0.01, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

NN = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=False)

model.save('chatbot.h5', NN)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# загружаем модель
model = load_model('chatbot.h5')
intents = json.loads(open("Train_Bot.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words



# возвращаем массив содержащий 0 или 1, показывающее, встретилось ли слово в предложении
def bow(sentence, words):

    sentence_words = clean_up_sentence(sentence)

    bag = [0] * len(words)

    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))


def predict_class(sentence, model):
    b = bow(sentence, words)
    result = model.predict(np.array([b]))[0]
    # threshold для обрезки незначимых значений
    error = 0.25
    results = [[i, res] for i, res in enumerate(result) if res > error]

    # сортируем по убыванию вероятностей
    results.sort(key=lambda x: x[1], reverse=True)


    return_list = []

    for res in results:
        return_list.append({"intent": classes[res[0]], "probability": str(res[1])})
    return return_list



def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res



# начинаем общение с ботом пока не напишем 'end'

def start_chat():
    print("Bot: This is ANDY! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

## <font color = 'green'> Пообщаемся с нашим ботом! </font>

In [10]:
start_chat()

Bot: This is ANDY! Your Personal Assistant.


Hello!
1/1 [==============================] - 0s 34ms/step
Bot: How are you?

--------------------------------------------------
Tell me about yourself
1/1 [==============================] - 0s 20ms/step
Bot: I am an artificial intelligence.

--------------------------------------------------
How old are you?
1/1 [==============================] - 0s 21ms/step
Bot: I have no age, I'm only getting newer every day.

--------------------------------------------------
Okay. Can you help me>
1/1 [==============================] - 0s 21ms/step
Bot: I'm glad to help. What can I do for you?

--------------------------------------------------
Do you like neural networks?
1/1 [==============================] - 0s 21ms/step
Bot: Neural Networks are cool! Part of my intelligence is NN too! Check https://en.wikipedia.org/wiki/Neural_network for more information 

--------------------------------------------------
NIce! Thanks, bye!
1/1 [================

# Источники
Идею взял отсюда: https://www.pycodemates.com/2021/11/build-a-AI-chatbot-using-python-and-deep-learning.html




---



---


При проверке работы бота были обнаружены некоторые "аномалии":
* Сообщения, содержащие 'are' скорее всего, будут ложно классифицированы как сообщения класса Annoying, так как 'are' чаще всего встречается в обучающей выборке именно в этом классе.